# Blocked reactions
Set the blocked reactions as model objective and use gapfilling of _E. coli_ panreactome to find reactions that can fill the gaps.

In [1]:
#import package needed
import cobra
import pandas as pd
from cobra.io import load_json_model
from glob import glob
from cobra.manipulation.delete import delete_model_genes
from cobra import Model, Reaction, Metabolite

### Model & media

In [2]:
#Establish a definition that initializes models to an in silico representation of M9 media
def m9(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0 
            
    model.reactions.EX_ca2_e.lower_bound=-1000
    model.reactions.EX_cl_e.lower_bound=-1000
    model.reactions.EX_co2_e.lower_bound=-1000
    model.reactions.EX_cobalt2_e.lower_bound=-1000
    model.reactions.EX_cu2_e.lower_bound=-1000
    model.reactions.EX_fe2_e.lower_bound=-1000
    model.reactions.EX_fe3_e.lower_bound=-1000
    model.reactions.EX_h_e.lower_bound=-1000
    model.reactions.EX_h2o_e.lower_bound=-1000
    model.reactions.EX_k_e.lower_bound=-1000
    model.reactions.EX_mg2_e.lower_bound=-1000
    model.reactions.EX_mn2_e.lower_bound=-1000
    model.reactions.EX_mobd_e.lower_bound=-1000
    model.reactions.EX_na1_e.lower_bound=-1000
    model.reactions.EX_tungs_e.lower_bound=-1000
    model.reactions.EX_zn2_e.lower_bound=-1000
    model.reactions.EX_ni2_e.lower_bound=-1000
    model.reactions.EX_sel_e.lower_bound=-1000
    model.reactions.EX_slnt_e.lower_bound=-1000
    model.reactions.EX_glc__D_e.lower_bound=-20
    model.reactions.EX_so4_e.lower_bound=-1000
    model.reactions.EX_nh4_e.lower_bound=-1000
    model.reactions.EX_pi_e.lower_bound=-1000
    model.reactions.EX_cbl1_e.lower_bound=-.01
    model.reactions.EX_o2_e.lower_bound=-20
       
    return model

In [3]:
# Load EcN model and iML1515 model
EcN_ID = 'CP022686.1'
model=cobra.io.load_json_model('../data/models/%s_cur_4.3.json'%EcN_ID)
base=cobra.io.load_json_model('../data/models/iML1515.json')
universal =cobra.io.load_json_model('../data/models/panecoli_universal.json')

# 1. Gapfilling blocked reactions

In [4]:
# The blocked reaction list was obtained from memote.io
blocked_reaction = ["EX_crn_e","HMPK1","LYSAM","EX_gthox_e","HEMEOS","UAG2E","DMQMT","OHPHM","SUCBZS","GOFUCR","PGLYCP","EDTXS3","EDTXS4","DMSOR1","DKGLCNR2y","2DGULRy","RZ5PP","EX_idon__L_e","EX_3hcinnm_e","EX_cbl1_e","DMSOR2","ACMAMUT","AB6PGH","GP4GH","ACGAMT","AADDGT","EX_gbbtn_e","GMHEPK","2DGULRx","G1PTT","TDPGDH","GMHEPPA","S7PI","CRNBTCT","CRNCBCT","CRNCDH","HETZK","TMAOR1","TMAOR2","DOGULNR","DKGLCNR1","ACPS1","AGMHE","GMHEPAT","SPMDAT1","SPMDAT2","TDPADGAT","EX_met__D_e","EX_mmet_e","ACBIPGT","ADOCBLS","ADOCBIK","CYANST","GMAND","DXYLK","GPDDA5","AP5AH","MI1PP","UDPGD","TDPAGTA","UACMAMO","BSORy","EX_cynt_e","BSORx","HCINNMtex","HPPPNtex","TSULabcpp","CBL1abcpp","GBBTNtex","IDONtex","FRULYStex","EX_cbi_e","EX_adocbl_e","CRNt7pp","PPPNtex","CBIuabcpp","METDtex","CRNtex","CYNTtex","EX_mal__D_e","GLCTR1","LIPACabcpp","EX_eca4colipa_e","EX_sel_e","EX_hg2_e","MALTATr","LIPOtex","MCPST","LIPAHT2ex","ECAP1pp","UDCPDPpp","ECAP2pp","CU1abcpp","HG2abcpp","LA4NTpp","PACALDt2rpp","PEAMNtex","PACALDtex","LIPOt2pp","INOSTt4pp","ACANTHAT","EX_tym_e","DOPAtex","4HOXPACDtex","CBLAT","ARABDI","AGt3","SARCOX","3KGK","EX_cd2_e","TUNGSabcpp","METDabcpp","ADOCBLabcpp","MMETtex","CD2tex","DHPTPE","ASR","ASO3tex","MANGLYCtex","EX_frulys_e","ECAtpp","O16GLCT2","EX_hacolipa_e","CBIAT","ACONMT","FHL","PPPNt2rpp","HG2tex","MTRPOX","NNDMBRT","SELNPS","CRNCAR","UDPGDC","EX_na1_e","EX_pppn_e","DKGLCNR2x","GLCATr","MALTptspp","YUMPS","EX_ag_e","EX_dopa_e","TYMtex","EX_pacald_e","UDPKAAT","ULA4NFT","UPLA4FNT","UPLA4FNF","ULA4Ntppi","HEPT1","HEPT2","EX_so2_e","EX_aso3_e","EX_fe3dhbzs_e","EX_peamn_e","EX_34dhpac_e","CUt3","HEPK1","CYSSADS","SO2tex","SO2tpp","SUCBZL","GDMANE","G3PIabcpp","LPLIPAL2ATG120","DURADx","CFAS160G","LPLIPAL2ATG140","LPLIPAL2ATG141","LPLIPAL2ATG160","LPLIPAL2ATG180","LPLIPAL2ATG161","LPLIPAL2ATG181","ALPATE160pp","ALPATG160pp","LPLIPAL2ATE120","LPLIPAL2ATE140","LPLIPAL2ATE160","LPLIPAL2ATE180","LPLIPAL2ATE141","LPLIPAL2ATE161","LPLIPAL2ATE181","TPRDCOAS","EX_4hoxpacd_e","EX_manglyc_e","34dhpactex","FE3DHBZStonex","ECAP3pp","EX_3hpppn_e","OMBZLM","2DOXG6PP","UDCPPtppi","EX_lipoate_e","DSBCGT","DSBGGT","DHNAOT4","HG2t3pp","COLIPAabcpp","ACONIs","NTP12","CFAS160E","CFAS180E","CFAS180G","GTHOXtex","DSBAO1","DSBAO2","DSBDR","TDSR1","NTP11","TDSR2","OMMBLHX3","EX_tungs_e","CTBTCAL2","EX_crn__D_e","AMMQLT8","CRNDCAL2","CRNCAL2","EX_psclys_e","TUNGStex","KDUI","MALDtex","MALDt2_2pp","5DGLCNtex","EX_5dglcn_e","PSCLYStex","SELGTHR","SELGTHR3","EX_slnt_e","MPTAT","MOADSUx","BMOCOS","BWCOS","BWCOGDS1","BWCOGDS2","DHNPTE","DHMPTR","CCGS","CDGR","CPH4S","SHCHCS3","EX_ttrcyc_e","EX_mincyc_e","SEPHCHCS","DHNCOAS","DHNCOAT","EX_doxrbcn_e","EX_fusa_e","EX_cm_e","EX_novbcn_e","MOCDS","EX_rfamp_e","O16A4COLIPAabctex","COLIPAabctex","BMOGDS1","MPTS","MOCOS","BMOGDS2","CLIPAabctex","SELtex","SELR","CDGS","SLNTtex","SELGTHR2","COLIPAKpp","MPTSS","CPMPS","LIPOCT","LIPOS","OCTNLL","LIPATPT","LIPAMPL","MMM","MOGDS","WCOS","ACOLIPAabctex","ECA4COLIPAabctex","COLIPAPabctex","2DGULRGx","2DGULRGy","ARBTptspp","ARBTtex","EX_arbt_e","DMSOtpp","13PPDH2","APCS","ThDPAT","APPLDHr","CRNDtex","SELabcpp","DHBSZ3FEabcpp","FE3DHBZS3R","2DGLCptspp","FEENTERES","PSURItex","OMPHHXy","Ftpp","ACOXT","2HESR","CHOLID","CHOLATEtpp","PHDA","TEO2M","CYSItpp","DHPTDNR","DHPTDNRN","PCNO","PUACGAMtr","DC6PDA","5DKGR","4HBZt3pp","LKDRA","METNA","XYHDL","2DGLCtex","E4PP","PUACGAMS","RU5PP","EX_dhps_e","EX_cs1_e","CBL1tonex","CBItonex","ARHGDx","PSURIK","Ftex","CYSItex","PNSPA","EX_sq_e","CS1tex","ADOCBLtonex","UDPGPT","OMMBLHXy","ACP1p","EX_4abzglu_e","4ABZGLUtex","MC6PH","SLNTabcpp","PSURItpp","CYSTA","FE3DCITR5","CUt2pp","RHMND","3HBZCT","CXSAMS","EX_2dglc_e","AMPNTAT","BIOMASS_Ec_iML1515_WT_75p37M","SALCNH","SALCtpp","SALCptspp","TYRTRS","PHETRS","THRTRS","TAG1PK","CHITPH","CHITOBpts","ASPTRS","ARGTRS","FEex","FEpp","ADOCBLPS","METTRS","DRIBtex","PENGt1","3hoxpactex","DXYLUDtex","ICITtex","AIRStex","HPAtex","SALCNtex","TAGtex","DARBtex","3PGtex","AEPtex","ACONCtex","OAAtex","TETtex","PACt1","6APAt1","GULNLtex","RAFFtex","2PGtex","GALAMtex","4HTHRtex","FOAMtex","4HOXPACt2pp","UPLA4NT","NPHS","DHACOAH","SLCYSS","SELtpp","SLNTtpp","HISTRS","THZPSN","OXPTNDH","ALATRS","NMNR","3OXCOAT","LYSTRS","PFOR","FLDR","OMPHHX","MMM2","4HTHRtrpp","PPRDNDH","GALAMPTSpp","ENLIPIDAt1ex","FMETTRS","DSCLCOCH","TRPTRS","OAAt2_2pp","GLYTRS","IMACTD","SELCYSS","GLCR1TRA1","GLCR1TRB1","GALR1TRA2","GALR1TRB2","GLCTR3","HEPKB2","HEPKA2","GALR1TRA1","GALR1TRB1","HEPT3","AMMQT8_2","URFGTT","GHBDHx","NICRNS","SBDH","UNK3","TRPTA","PDHcr","3HBCOAHL","VALTRS","ILETRS","ENLIPIDAt2ex","PROTRS","CYSTRS","ENTERH","FE3DHBZSabcpp","ADOCBLPP","LEUTRS","OMMBLHX","GLNTRS","GLCOAS","RNAMtpp","SERTRS2","SERTRS","ASNTRS","AH6PI","PPOR","HEPK2","GLCTR2","DRBK","DRIBtpp","UAG4Ei","DKFPS2","RAFH","RAFHpp","OPETDC","OPTCCL","CMCMSAD","DHPDO","CMHMI","2HH24DDH","4H2KPILY","3HOXPACt2pp","4HOXPACMOF","3HPAOX","HPA3MO","4H3NPACAM","PENAM","2PHACTE","PHACTE","35PHACTE","34PHACTE","3PHACTE","4PHACTE","PHACOAOR","3PHACOAOR","3NTYROXDApp","4HALDD","34DHALDD","4H3NALDD","G3PCT","CELLBpts","DHPTDCs","ENTERGLCT3","PRECLBTNSYN","PRECLBTNmate","CLBTNpeptidase","CLBTNtex"]
blocked_reaction = [ x for x in blocked_reaction if "EX_" not in x ]

In [ ]:
m9(model)
m9(base)
m9(universal)
c_dict = {}

with model:
    for reaction in blocked_reaction:
        
        # Change the model objective to the blocked reactions
        model.objective = model.reactions.get_by_id(reaction)

        # Perform gapfill (universal = X, with X is model, base or universal)
        gapfiller = cobra.flux_analysis.gapfilling.GapFiller(model, universal=universal, demand_reactions=False, integer_threshold=1e-12)
        gapfiller.model.solver.configuration.tolerances.feasibility = 1e-16
        gapfiller.model.solver.configuration.tolerances.integrality = 1e-16
        gapfiller.model.solver.configuration.tolerances.optimality = 1e-16

        try:
            result = gapfiller.fill()

            # If FBA & gapfiller feasible, add gapfill reactions to dictionary
            modelCopy = model.copy()
            genes = []

            for rxn in result[0]:
                genes.append(rxn.id)
                print(reaction, '> Reaction to add:', rxn)
                modelCopy.add_reaction(rxn)

            c_dict[reaction] = genes
    #         print(reaction, modelCopy.optimize().objective_value)
        except:
            print(reaction, '= infeasible')
            c_dict[reaction] = 'infeasible'

c_dict

HMPK1 = infeasible
LYSAM = infeasible
HEMEOS = infeasible
UAG2E > Reaction to add: TDPDRE: dtdp4d6dg_c --> dtdp4d6dm_c
UAG2E > Reaction to add: TDPDRR: dtdp4d6dm_c + h_c + nadph_c --> dtdprmn_c + nadp_c
UAG2E > Reaction to add: ECA4OALpp: colipa_p + eca4und_p --> eca4colipa_p + h_p + udcpdp_p
UAG2E > Reaction to add: GALT1: gicolipa_c + udpg_c --> gagicolipa_c + h_c + udp_c
UAG2E > Reaction to add: HEPT4: adphep_LD_c + gggagicolipa_c --> adp_c + colipa_c + h_c
UAG2E > Reaction to add: MOAT3C: ckdo_c + phphhlipa_c --> cmp_c + h_c + kphphhlipa_c
UAG2E > Reaction to add: RHAT1: dtdprmn_c + kphphhlipa_c --> dtdp_c + h_c + icolipa_c
DMQMT = infeasible
OHPHM = infeasible
SUCBZS = infeasible
GOFUCR = infeasible
PGLYCP = infeasible
EDTXS3 > Reaction to add: EX_lipa_cold_e: lipa_cold_e --> 
EDTXS4 > Reaction to add: EX_lipa_cold_e: lipa_cold_e --> 
DMSOR1 = infeasible
DKGLCNR2y = infeasible
2DGULRy = infeasible
RZ5PP = infeasible
DMSOR2 = infeasible
ACMAMUT > Reaction to add: TDPDRE: dtdp4d6dg_

UPLA4FNT > Reaction to add: MOAT3C: ckdo_c + phphhlipa_c --> cmp_c + h_c + kphphhlipa_c
UPLA4FNT > Reaction to add: RHAT1: dtdprmn_c + kphphhlipa_c --> dtdp_c + h_c + icolipa_c
UPLA4FNF > Reaction to add: EX_acolipa_e: acolipa_e --> 
UPLA4FNF > Reaction to add: TDPDRE: dtdp4d6dg_c --> dtdp4d6dm_c
UPLA4FNF > Reaction to add: TDPDRR: dtdp4d6dm_c + h_c + nadph_c --> dtdprmn_c + nadp_c
UPLA4FNF > Reaction to add: GALT1: gicolipa_c + udpg_c --> gagicolipa_c + h_c + udp_c
UPLA4FNF > Reaction to add: HEPT4: adphep_LD_c + gggagicolipa_c --> adp_c + colipa_c + h_c
UPLA4FNF > Reaction to add: MOAT3C: ckdo_c + phphhlipa_c --> cmp_c + h_c + kphphhlipa_c
UPLA4FNF > Reaction to add: RHAT1: dtdprmn_c + kphphhlipa_c --> dtdp_c + h_c + icolipa_c
ULA4Ntppi > Reaction to add: EX_acolipa_e: acolipa_e --> 
ULA4Ntppi > Reaction to add: TDPDRE: dtdp4d6dg_c --> dtdp4d6dm_c
ULA4Ntppi > Reaction to add: TDPDRR: dtdp4d6dm_c + h_c + nadph_c --> dtdprmn_c + nadp_c
ULA4Ntppi > Reaction to add: GALT1: gicolipa_c + u

In [ ]:
# obtain a dataframe with all reactions. Remove duplicates
gapfill_rxn_df = pd.DataFrame.from_dict(c_dict, orient='index', columns=['rxn_to_add']).astype({'rxn_to_add': str})
gapfill_rxn = gapfill_rxn_df.rxn_to_add.unique().tolist()

In [ ]:
gapfill_rxn

# 2. Reaction addition

When missing reactions are transport reactions of pathways that are further present, there were included in the model. For all reactions inside the cell, evidence from literature of databases was needed.

In [4]:
# Load EcN model and iML1515 model
EcN_ID = 'CP022686.1'
model = cobra.io.load_json_model('../data/models/%s_cur_4.3.json'%EcN_ID)
base = cobra.io.load_json_model('../data/models/iML1515.json')
LF82 = cobra.io.load_json_model('../data/models/iLF82_1304.json')

# Load the table with information on origin of reactions
rxn_origin_df = pd.read_csv('../tables/rxn_origin.csv')
rxn_origin_df.set_index('reaction', inplace=True)
rxn_origin_df.head()

,origin,added,notebook,notes
reaction,,,,
ALATA_D2,iML1515,automatic,3.1,none
SHCHD2,iML1515,automatic,3.1,none
CPPPGO,iML1515,automatic,3.1,none
GTHOr,iML1515,automatic,3.1,none
DHORD5,iML1515,automatic,3.1,none


### ['TDPDRE', 'TDPDRR', 'ECA4OALpp', 'HEPT4', 'MOAT3C', 'RHAT1', 'GALT1', '4ABZGLUtr']
All genes present in iML1515. 
rfb genes are part of the dTDP-β-L-rhamnose biosynthesis pathway, which synthesizes one of the precursors for the O antigen of lipopolysaccharide 
<br> TDPDRE> DTDP-4-dehydrorhamnose 3,5-epimerase, b2038 (rfbC)
<br> TDPDRR> DTDP-4-dehydrorhamnose reductase, b2040 (rfbD)
<br> ECA4OALpp> Enterobacterial common antigen (x4) O-antigen ligase (periplasm), b3622 (waaL)
<br> https://biocyc-org.proxy.findit.dtu.dk/gene?orgid=GCF_000714595&id=ECOLIN_RS20660
<br> HEPT4> Heptosyltransferase IV (LPS core synthesis), b3623 (waaU)
<br> MOAT3C> 3-deoxy-D-manno-octulosonic acid transferase III (LPS core biosynthesis), b3624 (waaZ)
<br> RHAT1> Rhamnosyltransferase I (LPS core biosynthesis), b3629 (waaS)
<br> GALT1> Galactosyltransferase I (LPS core synthesis), b3628 (waaB)
<br> 4ABZGLUtr> P-aminobenzoyl glutamate:H+ symporter, b1336 (abgT)

Nissle has specific semirough LPS (O6:K5:H1) https://jb.asm.org/content/jb/184/21/5912.full.pdf. This will not be adapted in this model, as this is not of high importance for predictions of recombinant metabolite production.

In [5]:
# Not added

### EX_lipa_cold_e
All reactions leading up to the export of lipa_cold_e are present in the model. It makes sense to also include the export reaction as sink of this metabolite

In [6]:
# Add reaction from LF82 strain
model.add_reactions([LF82.reactions.get_by_id('EX_lipa_cold_e')])

rxn_origin_df.loc['EX_lipa_cold_e', 'origin'] = 'iLF82_1304'
rxn_origin_df.loc['EX_lipa_cold_e', 'added'] = 'manual'
rxn_origin_df.loc['EX_lipa_cold_e', 'notebook'] = '4.4'
rxn_origin_df.loc['EX_lipa_cold_e', 'notes'] = 'Added in 4.4. to complete a blocked pathway'

# Transport > not linked to a gene

### INSTt2
There is a gap between INOSTt4pp and EX_inost_e. Other reactions in the pathway are present

In [7]:
# Add reaction from LF82 strain
model.add_reactions([LF82.reactions.get_by_id('INSTt2')])
model.reactions.INSTt2.subsystem = 'Transport, Outer Membrane'

rxn_origin_df.loc['INSTt2', 'origin'] = 'iLF82_1304'
rxn_origin_df.loc['INSTt2', 'added'] = 'manual'
rxn_origin_df.loc['INSTt2', 'notebook'] = '4.4'
rxn_origin_df.loc['INSTt2', 'notes'] = 'Added in 4.4 to complete a blocked pathway'

# Transport > not linked to a gene

### MMCD
Methylmalonyl-CoA decarboxylase
<br> Alternate Carbon Metabolism: b2919 (scpB)
<br> b2919,CIW80_15400,33.202,253,151,7,14,258,14,256,5.999999999999999e-37,128.0,262,0.9656488549618321,->

Many other steps of the pathway are present. However, couldn't find evidence in literature or databases for the existence of this reaction in EcN.

In [8]:
# Not added

### CYSIabcpp
L-cystine uptake via ABC system periplasm. Uptake was not tested in biolog data and no evidence could be found in literature. cysi__L_c is not converted in the cell, so reaction is not of importance.

In [9]:
# Not added

### 5DGLCNt2rpp
5-Dehydro-D-gluconate transport via proton symport, reversible (periplasm)
<br> Transport, Inner Membrane, b4265 (idnT)
<br> b4265,CIW80_11615,61.872,438,167,0,1,438,1,438,0.0,532.0,440,0.9954545454545455,->

Missing step in transport of 5-Dehydro-D-gluconate. Based on biolog this is indeed a carbon source it cannot use. 

In [10]:
# Not added

### RBK_Dr
Multiple gaps in pathway

In [11]:
# Not added

### PACCOAL
Missing link to 3PHACOAOR, but that reaction is also blocked

In [12]:
# Not added

# 3. Save updated model

In [13]:
# Save the model
cobra.io.json.save_json_model(model, str('../data/models/%s_cur_4.4.json'%EcN_ID), pretty=False)

In [14]:
# Save as a table
rxn_origin_df.to_csv('../tables/rxn_origin.csv')

# 4. Scenario add all reactions

In [15]:
with model:
    for rxn in ['TDPDRR', 'TDPDRE', 'ECA4OALpp', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1',
                'EX_lipa_cold_e','TDPDRR', 'TDPDRE', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1', 'CYTK2_2',
               'EX_acolipa_e', 'TDPDRR', 'TDPDRE', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1',
                'EX_colipa_e', 'TDPDRR', 'TDPDRE', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1',
                'INSTt2', 'EX_colipap_e', 'TDPDRR', 'TDPDRE', 'ECA4OALpp', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1',
                'MMCD', 'CYSIabcpp', '5DGLCNt2rpp', 'RBK_Dr', 'TDPDRR', 'TDPDRE', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1',
                'EX_acolipa_e', 'TDPDRR', 'TDPDRE', 'GALT1', 'HEPT4', 'MOAT3C', 'RHAT1', 'SK_thf_c', 'PACCOAL']:
        try:
            model.add_reaction(base.reactions.get_by_id(rxn))
        except:
            try:
                model.add_reaction(LF82.reactions.get_by_id(rxn))
            except:
                print(rxn, 'not added') # Several of the reactions are not present in the iML1515 model
            
    sol = model.optimize()
    
sol

Ignoring reaction 'EX_lipa_cold_e' since it already exists.
Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction 'RHAT1' since it already exists.


CYTK2_2 not added


Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction 'RHAT1' since it already exists.
Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction 'RHAT1' since it already exists.
Ignoring reaction 'INSTt2' since it already exists.
Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'ECA4OALpp' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction

RBK_Dr not added


Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction 'RHAT1' since it already exists.
Ignoring reaction 'EX_acolipa_e' since it already exists.
Ignoring reaction 'TDPDRR' since it already exists.
Ignoring reaction 'TDPDRE' since it already exists.
Ignoring reaction 'GALT1' since it already exists.
Ignoring reaction 'HEPT4' since it already exists.
Ignoring reaction 'MOAT3C' since it already exists.
Ignoring reaction 'RHAT1' since it already exists.


SK_thf_c not added


,fluxes,reduced_costs
ALATA_D2,0.00000,-2.105453e-02
SHCHD2,0.00025,0.000000e+00
CPPPGO,0.00025,0.000000e+00
GTHOr,0.00000,0.000000e+00
DHORD5,0.00000,-5.551115e-17
...,...,...
EX_colipap_e,0.00000,0.000000e+00
MMCD,0.00000,-1.263272e-02
CYSIabcpp,0.00000,-1.684362e-02
5DGLCNt2rpp,0.00000,0.000000e+00
